In [22]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [38]:
from proj1_helpers import *
from implementations import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [55]:
# Ridge regression with K-fold
k_fold= 2
degrees = range(1, 3)
lambdas = np.logspace(-4, 0, 30)
seed = 1
#ratio_train = 0.7

rmse_te = 0
w_BestDeg=0
BestDeg=0
#x_tr, x_te, y_tr, y_te= split_data(tX, y, ratio_train, seed)
k_indices =build_k_indices(y, k_fold, seed)
w_BestLambda=[]
rmse_te_tmp = []
for index_degree, degree in enumerate(degrees):
    rmse_te_tmp2 = []
    w_optLambda=[]
    for index_lambda, lambda_ in enumerate(lambdas):
        loss_te_tmp=0
        w_l=[]
        for k in range(k_fold):
            _, loss_te, w=cross_validation(y, tX, k_indices, k, lambda_, degree)
            loss_te_tmp= loss_te_tmp+loss_te
            w_l.append(w)
        loss_te_tmp=loss_te_tmp/k_fold
        rmse_te_tmp2.append(np.sqrt(2*loss_te_tmp))
        w_optLambda.append(np.mean(w_l))
    w_BestLambda.append(w_optLambda[np.argmin(rmse_te_tmp2)])
    rmse_te_tmp.append(min(rmse_te_tmp2))
w_BestDeg=w_BestLambda[np.argmin(rmse_te_tmp)]
BestDeg=degree[np.argmin(rmse_te_tmp)]
rmse_te=min(rmse_te_tmp)


loss=rmse_te
weights=w_BestDeg
print(BestDeg)


(array([ 7.41129971e-05, -7.25876016e-03, -6.22312892e-03, -5.48339759e-04,
       -4.96889493e-03,  4.54679096e-04, -2.33304762e-02,  3.38543375e-01,
       -2.22691847e-04,  5.65498841e-03, -2.12436712e-01,  9.44509063e-02,
        4.45075583e-02,  3.10067811e-03,  4.24527786e-04, -1.41616133e-03,
        7.41858176e-03, -1.21598860e-03,  1.93922442e-03,  3.67452378e-03,
        6.65162027e-04, -4.98729109e-04, -2.00941692e-01, -7.00750001e-05,
        6.08636704e-04, -2.22616089e-04, -2.07926940e-05, -6.40537175e-03,
       -9.97592827e-03, -6.50750689e-03, -1.26415101e-02, -1.26415101e-02,
       -1.26415101e-02, -1.26415101e-02, -1.26415101e-02, -1.26415101e-02,
       -1.26415101e-02, -1.26415101e-02, -1.26415101e-02, -1.26415101e-02,
       -1.26415101e-02, -1.26415101e-02, -1.26415101e-02, -1.26415101e-02,
       -1.26415101e-02, -1.26415101e-02, -1.26415101e-02, -1.26415101e-02,
       -1.26415101e-02, -1.26415101e-02, -1.26415101e-02, -1.26415101e-02,
       -1.26415101e-02, 

/Users/robindebalme/Documents/Cours EPFL MA1/Machine Learning/ml-project-1-pml/scripts/implementations.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  e = y - tx.dot(w)


ValueError: shapes (125000,60) and (2,) not aligned: 60 (dim 1) != 2 (dim 0)

In [ ]:
# Least squares regression using normal equation
weight, loss = least_squares(y, tX)
loss

0.33968680947709307

In [59]:
# Ridge regression using normal equations
lambda_ = 1e-3
tx=build_poly(tX,4)

weight, loss = ridge_regression(y, tx, lambda_)
loss

LinAlgError: Singular matrix

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = 'result.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)